In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Reading all data

In [2]:
# Read Items
item_df = pd.read_csv('data/train/item_data.csv', index_col='item_id')
item_df.head()

,item_price,category_1,category_2,category_3,product_type
item_id,,,,,
26880,4602,11,35,20,3040
54939,3513,12,57,85,6822
40383,825,17,8,279,1619
8777,2355,13,58,189,5264
113705,1267,17,39,151,10239


In [3]:
# Reading view logs
user_log_df = pd.read_csv('data/train/view_log.csv', parse_dates=['server_time'])
user_log_df.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


In [4]:
training_data = pd.read_csv('data/train/train.csv', index_col='impression_id', parse_dates=['impression_time'])
training_data.head()

,impression_time,user_id,app_code,os_version,is_4G,is_click
impression_id,,,,,,
c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [41]:
testing_data = pd.read_csv('data/test/test.csv', index_col='impression_id', parse_dates=['impression_time'])
testing_data.head()

,impression_time,user_id,app_code,os_version,is_4G
impression_id,,,,,
a9e7126a585a69a32bc7414e9d0c0ada,2018-12-13 07:44:00,44754,127,latest,1
caac14a5bf2ba283db7708bb34855760,2018-12-13 07:45:00,29656,44,latest,0
13f10ba306a19ce7bec2f3cae507b698,2018-12-13 07:46:00,25234,296,latest,1
39c4b4dc0e9701b55a0a4f072008fb3f,2018-12-13 07:47:00,22988,207,latest,1
bf5a572cca75f5fc67f4b14e58b11d70,2018-12-13 07:48:00,35431,242,latest,1


## Preparing different dataframes for extracting features

In [6]:
# Merging items and view_logs
user_logs = pd.merge(user_log_df, item_df, how='left', on='item_id').fillna(0)
user_logs.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,android,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0
1,2018-10-15 08:58:00,android,503590,74788,7640,1376.0,7.0,71.0,228.0,545.0
2,2018-10-15 08:58:00,android,573960,23628,128855,4544.0,4.0,38.0,62.0,5609.0
3,2018-10-15 08:58:00,android,121691,2430,12774,904.0,17.0,39.0,252.0,2740.0
4,2018-10-15 08:58:00,android,218564,19227,28296,2304.0,12.0,57.0,54.0,7422.0


In [7]:
user_logs_by_time = user_logs.set_index(['user_id','server_time']).sort_index()
user_logs_by_time.head()

device_type  session_id  item_id  item_price  \
user_id server_time                                                        
0       2018-10-18 14:27:00     android      268465    44459      3776.0   
        2018-10-18 14:27:00     android      268465    44459      3776.0   
        2018-10-18 14:43:00     android      268465    44459      3776.0   
        2018-10-18 14:46:00     android      268465    44459      3776.0   
        2018-10-18 14:46:00     android      268465    44459      3776.0   

                             category_1  category_2  category_3  product_type  
user_id server_time                                                            
0       2018-10-18 14:27:00        14.0        29.0       199.0        7363.0  
        2018-10-18 14:27:00        14.0        29.0       199.0        7363.0  
        2018-10-18 14:43:00        14.0        29.0       199.0        7363.0  
        2018-10-18 14:46:00        14.0        29.0       199.0        7363.0  
        2018-10-18 14:46:00        14.0        29.0       199.0        7363.0

In [19]:
detials = user_logs_by_session.loc[(0,268465), :].index
delta = detials[-1] - detials[0]
delta.total_seconds()

1200.0

In [9]:
user_logs_by_session = user_logs.set_index(['user_id','session_id','server_time']).sort_index()
user_logs_by_session.head()

device_type  item_id  item_price  \
user_id session_id server_time                                            
0       86373      2018-12-04 07:19:00     android   116073      4181.0   
        143955     2018-11-11 10:09:00     android    89373      2496.0   
                   2018-11-11 10:10:00     android    10379      1238.0   
                   2018-11-11 10:10:00     android    10379      1238.0   
                   2018-11-11 10:11:00     android    89373      2496.0   

                                        category_1  category_2  category_3  \
user_id session_id server_time                                               
0       86373      2018-12-04 07:19:00        17.0        39.0       151.0   
        143955     2018-11-11 10:09:00        17.0         8.0       113.0   
                   2018-11-11 10:10:00        16.0        17.0       272.0   
                   2018-11-11 10:10:00        16.0        17.0       272.0   
                   2018-11-11 10:11:00        17.0         8.0       113.0   

                                        product_type  
user_id session_id server_time                        
0       86373      2018-12-04 07:19:00         124.0  
        143955     2018-11-11 10:09:00       10025.0  
                   2018-11-11 10:10:00          60.0  
                   2018-11-11 10:10:00          60.0  
                   2018-11-11 10:11:00       10025.0

In [21]:
train_test_data = training_data.append(testing_data, sort=False).fillna(0)
train_test_data.head()

,impression_time,user_id,app_code,os_version,is_4G,is_click
impression_id,,,,,,
c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0.0
45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1.0
70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0.0
8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0.0
182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0.0


In [22]:
app_ad_history_action = training_data.set_index(['user_id','app_code','impression_time']).sort_index()
app_ad_history_action = app_ad_history_action[['is_click']]
app_ad_history_action.head()

is_click
user_id app_code impression_time              
0       207      2018-11-26 23:30:00         0
2       190      2018-11-19 20:49:00         0
                 2018-11-20 20:29:00         0
                 2018-11-20 20:53:00         0
                 2018-11-21 21:47:00         0

In [23]:
app_ad_history = train_test_data.set_index(['user_id','app_code','impression_time']).sort_index()
app_ad_history = app_ad_history[['is_click']]
app_ad_history.head()

is_click
user_id app_code impression_time              
0       207      2018-11-26 23:30:00       0.0
1       386      2018-12-13 19:49:00       0.0
2       190      2018-11-19 20:49:00       0.0
                 2018-11-20 20:29:00       0.0
                 2018-11-20 20:53:00       0.0

In [24]:
ad_history_action = training_data.set_index(['user_id','impression_time']).sort_index()
ad_history_action = ad_history_action[['is_click']]
ad_history_action.head()

is_click
user_id impression_time              
0       2018-11-26 23:30:00         0
2       2018-11-19 20:49:00         0
        2018-11-20 20:29:00         0
        2018-11-20 20:53:00         0
        2018-11-21 21:47:00         0

In [25]:
ad_history = train_test_data.set_index(['user_id','impression_time']).sort_index()
ad_history = ad_history[['is_click']]
ad_history.head()

is_click
user_id impression_time              
0       2018-11-26 23:30:00       0.0
1       2018-12-13 19:49:00       0.0
2       2018-11-19 20:49:00       0.0
        2018-11-20 20:29:00       0.0
        2018-11-20 20:53:00       0.0

## Functions to extract useful features

In [26]:
idealTime = pd.Timestamp('2018-11-01 00:00:00')
def last_ad_seen(row):
    try:
        records = ad_history.loc[row.user_id, :][:row.impression_time][-3:-2]
        last_seen = records.index.values[0] if len(records) > 0 else idealTime
        return (pd.Timestamp(row.impression_time) - last_seen).total_seconds()
    except KeyError:
        return (pd.Timestamp(row.impression_time) - idealTime).total_seconds()
    
def app_last_ad_seen(row):
    try:
        records = app_ad_history.loc[(row.user_id, row.app_code), :][:row.impression_time][-3:-2]
        last_seen = records.index.values[0] if len(records) > 0 else idealTime
        return (pd.Timestamp(row.impression_time) - last_seen).total_seconds()
    except KeyError:
        return (pd.Timestamp(row.impression_time) - idealTime).total_seconds()

In [27]:
def calculate_click_ratio(records):
    total_ads = records.shape[0]
    total_clicks = records.is_click.sum()
    return (total_clicks / total_ads) if total_ads != 0 else 0.0

def app_wise_click_ratio(row):
    try:
        records = app_ad_history_action.loc[(row.user_id, row.app_code), :][:row.impression_time][:-1]
        return calculate_click_ratio(records)
    except KeyError:
        return 0.0

def overall_click_ratio(row):
    try:
        records = ad_history_action.loc[row.user_id, :][:row.impression_time][:-1]
        return calculate_click_ratio(records)
    except KeyError:
        return 0.0

In [100]:
def last_active_session_length(user_id, session):
    session_details = user_logs_by_session.loc[(user_id,session), :].index
    return (session_details[-1] - session_details[0]).total_seconds()

def last_active_session(user_history, impression_time):
    ad_time = pd.Timestamp(impression_time)
    last_session_time = user_history.index.values[-1] if len(user_history) > 0 else idealTime
    return (ad_time - last_session_time).total_seconds()

In [111]:
def data_processing(row):
    user = row.user_id
    user_history = user_logs_by_time.loc[user,:][:row.impression_time]
    
    if user_history.item_id.count() == 0:
        row['total_items'] = 0
        row['unique_items'] = 0
        row['total_sessions'] = 0
        row['last_active_session'] = last_active_session(user_history, row.impression_time)
        row['session_length'] = 0
        row['mean_price'] = 0.0
        row['std_price'] = 0.0
        row['unique_product_type'] = 0
        row['freq_product_type'] = 0
        row['unique_cat1'] = 0
        row['freq_cat1'] = 0
        row['unique_cat2'] = 0
        row['freq_cat2'] = 0
        row['unique_cat3'] = 0
        row['freq_cat3'] = 0
        row['unique_device'] = 0
        row['freq_device'] = 0
        row['app_last_ad_seen'] = row['last_active_session']
        row['last_ad_seen'] = row['last_active_session']
        row['app_click_ratio'] = 0.0
        row['overall_click_ratio'] = 0.0
        
    else:
        row['total_items'] = user_history.item_id.count()
        row['unique_items'] = len(user_history.item_id.unique())
        row['total_sessions'] = len(user_history.session_id.unique())

        row['last_active_session'] = last_active_session(user_history, row.impression_time)
        row['session_length'] = last_active_session_length(user, user_history.session_id.values[-1])

        row['mean_price'] = user_history.item_price.mean()
        row['std_price'] = user_history.item_price.std()

        row['unique_product_type'] = len(user_history.product_type.unique())
        row['freq_product_type'] = user_history.product_type.value_counts().index[0]

        row['unique_cat1'] = len(user_history.category_1.unique())
        row['freq_cat1'] = user_history.category_1.value_counts().index[0]

        row['unique_cat2'] = len(user_history.category_1.unique())
        row['freq_cat2'] = user_history.category_1.value_counts().index[0]

        row['unique_cat3'] = len(user_history.category_1.unique())
        row['freq_cat3'] = user_history.category_1.value_counts().index[0]

        row['unique_device'] = len(user_history.device_type.unique())
        row['freq_device'] = user_history.device_type.value_counts().index[0]

        row['app_last_ad_seen'] = app_last_ad_seen(row)
        row['last_ad_seen'] = last_ad_seen(row)

        row['app_click_ratio'] = app_wise_click_ratio(row)
        row['overall_click_ratio'] = overall_click_ratio(row)
    
    return row

In [89]:
change_types = [
    ('last_active_session', 'int'),
    ('session_length', 'int'),
    ('freq_cat3', 'int'),
    ('freq_cat2', 'int'),
    ('freq_cat1', 'int'),
    ('app_last_ad_seen', 'int'),
    ('last_ad_seen', 'int'),
    ('freq_product_type', 'int'),
]
def extract_features(dataset):
    dataset = dataset.apply(data_processing, axis=1)
    for col, dtype in change_types:
        dataset[col] = dataset[col].astype(dtype)
    dataset.os_version = dataset.os_version.replace({'latest':0,'intermediate': 1, 'old':2})
    dataset.freq_device = dataset.freq_device.replace({'android':0, 'web': 1, 'iphone':2}).astype('int')
    return dataset

In [90]:
sample_testing_data = testing_data
sample_testing_data = extract_features(sample_testing_data)
sample_testing_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 14e40cf9f6b71d972a8cbf6836a230c4 to 539145030fb3600d26d6a64130310035
Data columns (total 26 columns):
impression_time        5 non-null datetime64[ns]
user_id                5 non-null int64
app_code               5 non-null int64
os_version             5 non-null int64
is_4G                  5 non-null int64
total_items            5 non-null int64
unique_items           5 non-null int64
total_sessions         5 non-null int64
last_active_session    5 non-null int64
session_length         5 non-null int64
mean_price             5 non-null float64
std_price              5 non-null float64
unique_product_type    5 non-null int64
freq_product_type      5 non-null int64
unique_cat1            5 non-null int64
freq_cat1              5 non-null int64
unique_cat2            5 non-null int64
freq_cat2              5 non-null int64
unique_cat3            5 non-null int64
freq_cat3              5 non-null int64
unique_device          5 non-

## Extracting Features

In [97]:
testing_data = pd.read_csv('data/test/test.csv', index_col='impression_id', parse_dates=['impression_time'])
testing_data = extract_features(testing_data)
testing_data.head()

,impression_time,user_id,app_code,os_version,is_4G,total_items,unique_items,total_sessions,last_active_session,session_length,...,unique_cat2,freq_cat2,unique_cat3,freq_cat3,unique_device,freq_device,app_last_ad_seen,last_ad_seen,app_click_ratio,overall_click_ratio
impression_id,,,,,,,,,,,,,,,,,,,,,
a9e7126a585a69a32bc7414e9d0c0ada,2018-12-13 07:44:00,44754,127,0,1,1,1,1,132180,0,...,1,1,1,1,1,0,49980,49980,0.000000,0.000000
caac14a5bf2ba283db7708bb34855760,2018-12-13 07:45:00,29656,44,0,0,72,46,25,145620,0,...,14,1,14,1,1,0,1128000,1128000,0.000000,0.000000
13f10ba306a19ce7bec2f3cae507b698,2018-12-13 07:46:00,25234,296,0,1,9,7,6,549000,0,...,5,7,5,7,1,0,3656760,3656760,0.000000,0.000000
39c4b4dc0e9701b55a0a4f072008fb3f,2018-12-13 07:47:00,22988,207,0,1,37,17,12,259800,0,...,11,9,11,9,1,0,815700,815700,0.000000,0.000000
bf5a572cca75f5fc67f4b14e58b11d70,2018-12-13 07:48:00,35431,242,0,1,28,26,18,471120,0,...,12,11,12,11,1,0,818460,818460,0.210526,0.210526


In [98]:
testing_data.to_csv('data/test/test_feature.csv')

In [112]:
training_data = pd.read_csv('data/train/train.csv', index_col='impression_id', parse_dates=['impression_time'])
training_data = extract_features(training_data)
training_data.head()

,impression_time,user_id,app_code,os_version,is_4G,is_click,total_items,unique_items,total_sessions,last_active_session,...,unique_cat2,freq_cat2,unique_cat3,freq_cat3,unique_device,freq_device,app_last_ad_seen,last_ad_seen,app_click_ratio,overall_click_ratio
impression_id,,,,,,,,,,,,,,,,,,,,,
c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,2,0,0,1,1,1,2267340,...,1,11,1,11,1,0,1209600,1209600,0.0,0.0
45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,0,1,1,7,3,5,1796280,...,2,4,2,4,1,0,1209660,1209660,0.0,0.0
70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,1,1,0,1,1,1,1940580,...,1,13,1,13,1,0,1209720,1209720,0.0,0.0
8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,0,1,0,11,6,3,18060,...,5,14,5,14,1,0,1209720,1209720,0.0,0.0
182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,0,0,0,9,8,7,8400,...,6,11,6,11,1,0,1209720,1209720,0.0,0.0


In [113]:
training_data.to_csv('data/train/train_feature.csv')